In [1]:
import os
import sys
import random
os.environ["KERAS_BACKEND"] = "tensorflow"

import glob
try:
    if not ("CUDA_VISIBLE_DEVICES" in os.environ):
        os.environ['CUDA_VISIBLE_DEVICES']='0'
        print("importing setGPU")
        import setGPU
except:
    print("Could not import setGPU, please make sure you configure CUDA_VISIBLE_DEVICES manually")
    pass

import pickle
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import confusion_matrix, accuracy_score
import pandas
import time
from tqdm import tqdm
import itertools
import io
import sklearn
import sklearn.cluster
import tensorflow as tf
from numpy.lib.recfunctions import append_fields

import scipy
import scipy.special

from mpnn import MessagePassing, ReadoutGraph, Aggregation

importing setGPU
Could not import setGPU, please make sure you configure CUDA_VISIBLE_DEVICES manually


In [2]:
import tensorflow as tf
import json, os
import numpy as np

# Tested with TensorFlow 2.1.0
print('version={}, CUDA={}, GPU={}, TPU={}'.format(
    tf.__version__, tf.test.is_built_with_cuda(),
    # GPU attached?
    len(tf.config.list_physical_devices('GPU')) > 0,
    # TPU accessible? (only works on Colab)
    'COLAB_TPU_ADDR' in os.environ))

version=2.1.0, CUDA=True, GPU=True, TPU=False


In [3]:
def dist(A,B):
    na = tf.reduce_sum(tf.square(A), -1)
    nb = tf.reduce_sum(tf.square(B), -1)
 
    na = tf.reshape(na, [tf.shape(na)[0], -1, 1])
    nb = tf.reshape(nb, [tf.shape(na)[0], 1, -1])
    Dsq = tf.clip_by_value(na - 2*tf.linalg.matmul(A, B, transpose_a=False, transpose_b=True) + nb, 1e-12, 1e12)
    D = tf.sqrt(Dsq)
    return D


In [4]:
#Given a list of [Nbatch, Nelem, Nfeat] input nodes, computes the dense [Nbatch, Nelem, Nelem] adjacency matrices
class Distance(tf.keras.layers.Layer):

    def __init__(self, dist_shape, *args, **kwargs):
        super(Distance, self).__init__(*args, **kwargs)

    def call(self, inputs1, inputs2):
        #compute the pairwise distance matrix between the vectors defined by the first two components of the input array
        #inputs1, inputs2: [Nbatch, Nelem, distance_dim] embedded coordinates used for element-to-element distance calculation
        D = dist(inputs1, inputs2)
      
        #adjacency between two elements should be high if the distance is small.
        #this is equivalent to radial basis functions. 
        #self-loops adj_{i,i}=1 are included, as D_{i,i}=0 by construction
        adj = tf.math.exp(-1.0*D)

        #optionally set the adjacency matrix to 0 for low values in order to make the matrix sparse.
        #need to test if this improves the result.
        #adj = tf.keras.activations.relu(adj, threshold=0.01)

        return adj

In [5]:
class InputEncoding(tf.keras.layers.Layer):
    def __init__(self, num_input_classes):
        super(InputEncoding, self).__init__()
        self.num_input_classes = num_input_classes
        
    def call(self, X):
        #X: [Nbatch, Nelem, Nfeat] array of all the input detector element feature data

        #X[:, :, 0] - categorical index of the element type
        Xid = tf.one_hot(tf.cast(X[:, :, 0], tf.int32), self.num_input_classes)

        #X[:, :, 1:] - all the other non-categorical features
        Xprop = X[:, :, 1:]
        return tf.concat([Xid, Xprop], axis=-1)

In [6]:
## Graph Highway network
# https://arxiv.org/pdf/2004.04635.pdf
#https://github.com/gcucurull/jax-ghnet/blob/master/models.py 
class GHConv(tf.keras.layers.Layer):
    def __init__(self, k, *args, **kwargs):
        self.activation = kwargs.pop("activation")
        self.hidden_dim = args[0]
        self.k = k

        super(GHConv, self).__init__(*args, **kwargs)

        self.W_t = self.add_weight(shape=(self.hidden_dim, self.hidden_dim), name="w_t", initializer="random_normal")
        self.b_t = self.add_weight(shape=(self.hidden_dim, ), name="b_t", initializer="zeros")
        self.W_h = self.add_weight(shape=(self.hidden_dim, self.hidden_dim), name="w_h", initializer="random_normal")
        self.theta = self.add_weight(shape=(self.hidden_dim, self.hidden_dim), name="theta", initializer="random_normal")
 
    def call(self, x, adj):
        #compute the normalization of the adjacency matrix
        in_degrees = tf.reduce_sum(adj, axis=-1)
        #add epsilon to prevent numerical issues from 1/sqrt(x)
        norm = tf.expand_dims(tf.pow(in_degrees + 1e-6, -0.5), -1)
        norm_k = tf.pow(norm, self.k)
        adj_k = tf.pow(adj, self.k)

        f_hom = tf.linalg.matmul(x, self.theta)
        f_hom = tf.linalg.matmul(adj_k, f_hom*norm_k)*norm_k

        f_het = tf.linalg.matmul(x, self.W_h)
        gate = tf.nn.sigmoid(tf.linalg.matmul(x, self.W_t) + self.b_t)
        #tf.print(tf.reduce_mean(f_hom), tf.reduce_mean(f_het), tf.reduce_mean(gate))

        out = gate*f_hom + (1-gate)*f_het
        return out

## Simple Graph Conv layer
class SGConv(tf.keras.layers.Dense):
    def __init__(self, k, *args, **kwargs):
        super(SGConv, self).__init__(*args, **kwargs)
        self.k = k
    
    def call(self, inputs, adj):
        W = self.weights[0]
        b = self.weights[1]

        #compute the normalization of the adjacency matrix
        in_degrees = tf.reduce_sum(adj, axis=-1)
        #add epsilon to prevent numerical issues from 1/sqrt(x)
        norm = tf.expand_dims(tf.pow(in_degrees + 1e-6, -0.5), -1)
        norm_k = tf.pow(norm, self.k)

        support = (tf.linalg.matmul(inputs, W))
     
        #k-th power of the normalized adjacency matrix is nearly equivalent to k consecutive GCN layers
        adj_k = tf.pow(adj, self.k)
        out = tf.linalg.matmul(adj_k, support*norm_k)*norm_k

        return self.activation(out + b)

In [108]:
#Simple message passing based on a matrix multiplication
class DNNSuperCluster(tf.keras.Model):
    
    def __init__(self, activation=tf.nn.selu, nclass_labels=2, hidden_dim=256, distance_dim=256, num_conv=4, convlayer="ghconv", dropout=0.1):
        super(DNNSuperCluster, self).__init__()
        self.activation = activation
        self.nclass_labels = nclass_labels

        #self.enc = InputEncoding(3)

        self.layer_distcoords1 = tf.keras.layers.Dense(hidden_dim, activation=activation, name="distcoords1")
        self.layer_distcoords2 = tf.keras.layers.Dense(hidden_dim, activation=activation, name="distcoords2")
        self.layer_distcoords3 = tf.keras.layers.Dense(hidden_dim, activation=activation, name="distcoords3")
        self.layer_distcoords = tf.keras.layers.Dense(distance_dim, activation="linear", name="distcoords")

        self.layer_input1 = tf.keras.layers.Dense(hidden_dim, activation=activation, name="input1")
        self.layer_input1_do = tf.keras.layers.Dropout(dropout)
        self.layer_input2 = tf.keras.layers.Dense(hidden_dim, activation=activation, name="input2")
        self.layer_input2_do = tf.keras.layers.Dropout(dropout)
        self.layer_input3 = tf.keras.layers.Dense(2*hidden_dim, activation=activation, name="input3")
        self.layer_input3_do = tf.keras.layers.Dropout(dropout)
        
            
        self.layer_dist = Distance(distance_dim, name="distance")

        if convlayer == "sgconv":
            self.layer_conv1 = SGConv(num_conv, 2*hidden_dim, activation=activation, name="conv1")
            #self.layer_conv2 = SGConv(num_conv, 2*hidden_dim+len(class_labels), activation=activation, name="conv2")
        elif convlayer == "ghconv":
            self.layer_conv1 = GHConv(num_conv, 2*hidden_dim, activation=activation, name="conv1")
            #self.layer_conv2 = GHConv(num_conv, 2*hidden_dim+len(class_labels), activation=activation, name="conv2")

        self.layer_id1 = tf.keras.layers.Dense(2*hidden_dim, activation=activation, name="id1")
        self.layer_id2 = tf.keras.layers.Dense(hidden_dim, activation=activation, name="id2")
        self.layer_id3 = tf.keras.layers.Dense(hidden_dim, activation=activation, name="id3")
        self.layer_id = tf.keras.layers.Dense(nclass_labels, activation="linear", name="out_id")
        #self.layer_charge = tf.keras.layers.Dense(1, activation="linear", name="out_charge")
        
        #self.layer_momentum1 = tf.keras.layers.Dense(2*hidden_dim, activation=activation, name="momentum1")
        #self.layer_momentum2 = tf.keras.layers.Dense(hidden_dim, activation=activation, name="momentum2")
        #self.layer_momentum3 = tf.keras.layers.Dense(hidden_dim, activation=activation, name="momentum3")
        #self.layer_momentum = tf.keras.layers.Dense(3, activation="linear", name="out_momentum")
 
    def predict_distancematrix(self, inputs, training=True):

        act_inputs = self.activation(inputs)
        x = self.layer_distcoords1(inputs)
        x = self.layer_distcoords2(x)
        x = self.layer_distcoords3(x)
        distcoords = self.layer_distcoords(x)

        dm = self.layer_dist(distcoords, distcoords)
        
        # masking if the first element is -1
        msk_elem = tf.expand_dims(tf.cast(inputs[:, :, 0] != -1, dtype=tf.float32), -1)
        dm = dm*msk_elem

        return act_inputs, dm

    #@tf.function(input_signature=[tf.TensorSpec(shape=[None, 15], dtype=tf.float32)])
    def call(self, inputs, training=True):
        X = inputs
        msk_input = tf.expand_dims(tf.cast(X[:, :, 0] != -1, tf.float32), -1)

        act_inputs, dm = self.predict_distancematrix(X, training=training)

        x = self.layer_input1(act_inputs)
        x = self.layer_input1_do(x, training)
        x = self.layer_input2(x)
        x = self.layer_input2_do(x, training)
        x = self.layer_input3(x)
        x = self.layer_input3_do(x, training)
        x = self.layer_conv1(x, dm)
        x = self.layer_id1(x)
        x = self.layer_id2(x)
        x = self.layer_id3(x)
        out_id_logits = self.layer_id(x)


        ret = out_id_logits*msk_input
        return ret

In [8]:
class DNNSuperClusterOld(tf.keras.Model):
    
    def __init__(self, activation=tf.keras.activations.relu, hidden_dim_in=256,hidden_dim_out=256):
        super(DNNSuperCluster, self).__init__()
        self.layer_input1 = tf.keras.layers.Dense(hidden_dim_in, activation=activation, name="input1")
        self.layer_input2 = tf.keras.layers.Dense(hidden_dim_in, activation=activation, name="input2")
        self.layer_input3 = tf.keras.layers.Dense(hidden_dim_in, activation=activation, name="input3")
        
        self.layer_dist = Distance(name="distance")
        self.layer_conv = GraphConv(hidden_dim_in, activation=activation, name="conv")
        
        self.layer_id1 = tf.keras.layers.Dense(hidden_dim_out, activation=activation, name="id1")
        self.layer_id2 = tf.keras.layers.Dense(hidden_dim_out, activation=activation, name="id2")
        self.layer_id3 = tf.keras.layers.Dense(hidden_dim_out, activation=activation, name="id3")
        self.layer_id = tf.keras.layers.Dense(1, activation="sigmoid", name="out_id")
        
        
    def call(self, inputs):
        #print(inputs)
        x_init = tf.concat([inputs[0],tf.broadcast_to(inputs[1],[inputs[0].shape[0], inputs[1].shape[1]] )],
                  axis=1)
        
        x = self.layer_input1(x_init)
        x = self.layer_input2(x)
        x_before_gcn = self.layer_input3(x)
        
        dm = self.layer_dist(x_before_gcn)
        #print("distance matrix:", dm.shape)
        x_gcn = self.layer_conv(x_before_gcn, dm)
        
        #print(x.shape)
        x_join = tf.concat([x_init, x_gcn], axis=1)
        
        a = self.layer_id1(x_join)
        a = self.layer_id2(a)
        a = self.layer_id3(a)
        out_id = self.layer_id(a)
    
        
        return out_id, dm

# Loss definition

In [9]:
def mse_unreduced(true, pred):
    return tf.math.pow(true-pred,2)

def msle_unreduced(true, pred):
    return tf.math.pow(tf.math.log(tf.math.abs(true) + 1.0) - tf.math.log(tf.math.abs(pred) + 1.0), 2)

def my_loss_cls(y_true, y_pred):
    pred_id_onehot, pred_charge, pred_momentum = separate_prediction(y_pred)
    pred_id = tf.cast(tf.argmax(pred_id_onehot, axis=-1), tf.int32)
    true_id, true_charge, true_momentum = separate_truth(y_true)

    true_id_onehot = tf.one_hot(tf.cast(true_id, tf.int32), depth=len(class_labels))
    #predict the particle class labels
    l1 = 1e4*tf.nn.softmax_cross_entropy_with_logits(true_id_onehot, pred_id_onehot)
    #l1 = 1e4*tf.keras.losses.categorical_crossentropy(true_id_onehot[:, :, 0], pred_id_onehot, from_logits=True)
    return 1e3*l1

def my_loss_reg(y_true, y_pred):
    pred_id_onehot, pred_charge, pred_momentum = separate_prediction(y_pred)
    pred_id = tf.cast(tf.argmax(pred_id_onehot, axis=-1), tf.int32)
    true_id, true_charge, true_momentum = separate_truth(y_true)

    true_id_onehot = tf.one_hot(tf.cast(true_id, tf.int32), depth=len(class_labels))

    l2_0 = mse_unreduced(true_momentum[:, :, 0], pred_momentum[:, :, 0])*10
    l2_1 = mse_unreduced(tf.math.floormod(true_momentum[:, :, 1] - pred_momentum[:, :, 1] + np.pi, 2*np.pi) - np.pi, 0.0)*10
    l2_2 = mse_unreduced(true_momentum[:, :, 2], pred_momentum[:, :, 2])/100.0

    l2 = (l2_0 + l2_1 + l2_2)
    
    return 1e3*l2

#@tf.function
def my_loss_full(y_true, y_pred):
    pred_id_onehot, pred_charge, pred_momentum = separate_prediction(y_pred)
    pred_id = tf.cast(tf.argmax(pred_id_onehot, axis=-1), tf.int32)
    true_id, true_charge, true_momentum = separate_truth(y_true)

    true_id_onehot = tf.one_hot(tf.cast(true_id, tf.int32), depth=len(class_labels))
    #tf.print(pred_id_onehot)
    l1 = 1e3*tf.nn.softmax_cross_entropy_with_logits(true_id_onehot, pred_id_onehot)
  
    #msk_good = (true_id[:, 0] == pred_id)
    #nsamp = tf.cast(tf.size(y_pred), tf.float32)

    l2_0 = mse_unreduced(true_momentum[:, :, 0], pred_momentum[:, :, 0])
    l2_1 = mse_unreduced(tf.math.floormod(true_momentum[:, :, 1] - pred_momentum[:, :, 1] + np.pi, 2*np.pi) - np.pi, 0.0)
    l2_2 = mse_unreduced(true_momentum[:, :, 2], pred_momentum[:, :, 2])/100.0

    l2 = (l2_0 + l2_1 + l2_2)
    #l2 = tf.multiply(tf.cast(msk_good, tf.float32), l2)

    l3 = mse_unreduced(true_charge, pred_charge)[:, :, 0]

    #tf.debugging.check_numerics(l1, "l1")
    #tf.debugging.check_numerics(l2_0, "l2_0")
    #tf.debugging.check_numerics(l2_1, "l2_1")
    #tf.debugging.check_numerics(l2_2, "l2_2")

    #tf.print("l1", tf.reduce_mean(l1))
    #tf.print("l2_0", tf.reduce_mean(l2_0))
    #tf.print("l2_1", tf.reduce_mean(l2_1))
    #tf.print("l2_2", tf.reduce_mean(l2_2))
    #tf.print("l2", tf.reduce_mean(l2))
    #tf.print("l3", tf.reduce_mean(l3))

    #tf.print("\n")
    l = l1 + l2 + l3
    return 1e3*l


In [10]:
tf.config.experimental_run_functions_eagerly(True)

In [11]:
try:
    num_gpus = len(os.environ["CUDA_VISIBLE_DEVICES"].split(","))
    print("num_gpus=", num_gpus)
    if num_gpus > 1:
        strategy = tf.distribute.MirroredStrategy()
    else:
        strategy = tf.distribute.OneDeviceStrategy("gpu:0")
except Exception as e:
    print(e)
    print("fallback to CPU")
    strategy = tf.distribute.OneDeviceStrategy("cpu")

num_gpus= 1


## Dataset loading

In [74]:
n_features = 15
n_seed_features = 6

def _parse_tfr_element(element):
    parse_dic = {
        'X':      tf.io.FixedLenFeature([], tf.string),
        'X_seed': tf.io.FixedLenFeature([], tf.string),
        'y':      tf.io.FixedLenFeature([], tf.string),
    }
    example_message = tf.io.parse_single_example(element, parse_dic)

    X = example_message['X']
    X_seed = example_message['X_seed']
    y = example_message['y']
    arr_X = tf.io.parse_tensor(X, out_type=tf.float32)
    arr_X_seed = tf.io.parse_tensor(X_seed, out_type=tf.float32)
    arr_y = tf.io.parse_tensor(y, out_type=tf.float32)
    
    #https://github.com/tensorflow/tensorflow/issues/24520#issuecomment-577325475
    arr_X.set_shape(     tf.TensorShape((None, n_features)))
    arr_X_seed.set_shape(tf.TensorShape((1, n_seed_features)))
    arr_y.set_shape(     tf.TensorShape((None,)))
    
    arr_y = tf.one_hot(tf.cast(arr_y[:], tf.int32), 2)
    
    return arr_X, arr_X_seed, arr_y
    

### Settings

In [105]:
data_path = "/storage/ECAL/training_data/window_data/gammas/recordio_v2"
models_path = "/storage/ECAL/deepcluster/models/gcn_models_v2"

#rain_steps_per_epoch = 
#eval_steps_per_epoch = 3e5 // batch_size
from collections import namedtuple
Args = namedtuple('args', [ 'models_path', 'nepochs','ntrain','nval','nfeatures',
                            'n_seed_features','batch_size','lr_decay','lr',
                            'nhidden','distance_dim','num_conv','dropout','convlayer',
                           'nclass_labels', 'opt'])

args = Args( 
        models_path = models_path,
        nepochs = 10,
        ntrain = 1000,
        nval = 100,
        batch_size = 52,
        nfeatures = 15,
        n_seed_features = 6,
        lr_decay = False,
        lr = 0.001,
        nhidden = 100,
        distance_dim = 100,
        num_conv = 4,
        dropout = 0.0,
        convlayer = 'sgconv',
        nclass_labels=2,
        opt='adam'
        )

In [112]:
# padding shape
ps = ([None,args.nfeatures],[1, args.n_seed_features],[None,2])

# Create datasets from TFRecord files.
dataset = tf.data.TFRecordDataset(tf.io.gfile.glob('{}/training-*'.format(data_path))).map(_parse_tfr_element,num_parallel_calls=tf.data.experimental.AUTOTUNE)

ds_train = dataset.take(ntrain).padded_batch(args.batch_size, padded_shapes=ps, drop_remainder=True,padding_values=(-1.,-1.,-1.))
ds_test = dataset.skip(ntrain).take(nval).padded_batch(args.batch_size, padded_shapes=ps, drop_remainder=True,padding_values=(-1.,-1.,-1.))

ds_train_r = ds_train.repeat(args.nepochs)
ds_test_r = ds_test.repeat(args.nepochs)


In [113]:
idata = iter(ds_train)
d = next(idata)

In [95]:
def get_unique_run():
    previous_runs = os.listdir(args.models_path)
    if len(previous_runs) == 0:
        run_number = 1
    else:
        run_number = max([int(s.split('run_')[1]) for s in previous_runs]) + 1
    return run_number

In [96]:
if args.lr_decay > 0:
        lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
            args.lr,
            decay_steps=10*int(args.ntrain/batch_size),
            decay_rate=args.lr_decay
        )
else:
    lr_schedule = args.lr


In [101]:
with strategy.scope():
    opt = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
    
    model = DNNSuperCluster(hidden_dim=args.nhidden, nclass_labels=args.nclass_labels, distance_dim=args.distance_dim, 
                            num_conv=args.num_conv, convlayer=args.convlayer, dropout=args.dropout)
   

In [103]:
if not os.path.isdir(args.models_path):
    os.makedirs(args.models_path)

name =  'run_{:02}'.format(get_unique_run())

outdir = args.models_path + name

if os.path.isdir(outdir):
    print("Output directory exists: {}".format(outdir), file=sys.stderr)
    sys.exit(1)

print(outdir)


/storage/ECAL/deepcluster/models/gcn_models_v2run_01


In [106]:
callbacks = []
tb = tf.keras.callbacks.TensorBoard(
    log_dir=outdir, histogram_freq=10, write_graph=True, write_images=False,
    update_freq='epoch',
    #profile_batch=(10,90),
    profile_batch=0,
)
tb.set_model(model)
callbacks += [tb]

terminate_cb = tf.keras.callbacks.TerminateOnNaN()
callbacks += [terminate_cb]

cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=outdir + "/weights.{epoch:02d}-{val_loss:.6f}.hdf5",
    save_weights_only=True,
    verbose=0
)
cp_callback.set_model(model)
callbacks += [cp_callback]

loss_fn = my_loss_full



In [107]:
with strategy.scope():
    model.compile(optimizer=args.opt, loss=loss_fn,
        metrics=[],
        sample_weight_mode="temporal")

    for X, y, w in ds_train:
        ypred = model(X)
        l = loss_fn(y, ypred)

    

NameError: name 'enc' is not defined

In [ ]:
if args.load:
        #ensure model input size is known
        for X, y, w in ds_train:
            model(X)
            break

        model.load_weights(args.load)
    if args.nepochs > 0:
        ret = model.fit(ds_train_r,
            validation_data=ds_test_r, epochs=args.nepochs,
            steps_per_epoch=args.ntrain/batch_size, validation_steps=args.ntest/batch_size,
            verbose=True,
            callbacks=callbacks
        )

In [21]:
def loss(model, inputs, targets):
    out, _ = model(inputs)
     # modified cross entropy to explicit mathematical formula of sigmoid cross entropy loss
    cross_entropy = - tf.reduce_sum( (  (targets[:,0]*tf.math.log(out + 1e-9)) + ((1-targets[:,0]) * tf.math.log(1 - out + 1e-9)) )  , name='xentropy' ) 
    
    return cross_entropy

In [33]:
d = next(idata)
loss(model, d[:2],d[2])

<tf.Tensor: shape=(), dtype=float32, numpy=71.99879>

In [34]:
def grad(model, inputs, targets, epoch):
    with tf.GradientTape() as tape:
        loss_value = loss(model, inputs, targets)
        return loss_value, tape.gradient(loss_value, model.trainable_variables)

In [ ]:
accum_vars = [tf.Variable(tf.zeros_like(tv.initialized_value()), trainable=False) for tv in model.trainable_variables]

for i in range(10):
    loss_tot = 0.0
    losses = []
    
    ibatch = 0
    
    true_y = []
    pred_y = []
    
    for ielem,train_data  in enumerate(train_ds):
        
        loss_value, grads = grad(model, train_data[:2],train_data[2], i)
        
        for igrad, gv in enumerate(grads):
            accum_vars[igrad].assign_add(gv)
        
        loss_tot += loss_value.numpy()
        if ibatch == 10:
            opt.apply_gradients([(accum_vars[igrad] / 10, model.trainable_variables[igrad]) for igrad in range(len(accum_vars))])
            ibatch = 0
            for igrad in range(len(accum_vars)):
                accum_vars[igrad].assign(tf.zeros_like(accum_vars[igrad]))
                
            losses.append(loss_tot)
            loss_tot = 0
        ibatch+=1

        if ielem % 250 == 0: 
            print(f"i: {ielem}, loss_tot: {np.mean(losses)}")
            losses.clear()
        
   

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


i: 0, loss_tot: nan
i: 250, loss_tot: 208.95696445910843
i: 500, loss_tot: 285.9967866801098
i: 750, loss_tot: 215.49208974038717
i: 1000, loss_tot: 235.45319188342896
i: 1250, loss_tot: 250.05417004236486
i: 1500, loss_tot: 270.592827648744
i: 1750, loss_tot: 239.17497981898487
i: 2000, loss_tot: 227.97677876235917
i: 2250, loss_tot: 246.68653825044632
i: 2500, loss_tot: 213.3957731453888
i: 2750, loss_tot: 225.02449972154574
i: 3000, loss_tot: 242.53036470352671
i: 3250, loss_tot: 235.186609955905
i: 3500, loss_tot: 240.67808114275337
i: 3750, loss_tot: 219.18794876560568
i: 4000, loss_tot: 186.38467956393959
i: 4250, loss_tot: 198.93314678423107
i: 4500, loss_tot: 224.8088186214678
i: 4750, loss_tot: 216.3698060000874
i: 5000, loss_tot: 212.80465302586555
i: 5250, loss_tot: 223.1398355620727
i: 5500, loss_tot: 237.8537559848884
i: 5750, loss_tot: 226.61613926644438
i: 6000, loss_tot: 265.59703569304196
i: 6250, loss_tot: 224.47560287389905
i: 6500, loss_tot: 297.88556430969385
i: 67

i: 53500, loss_tot: 196.63222378241247
i: 53750, loss_tot: 243.57906349463389
i: 54000, loss_tot: 194.37196313572232
i: 54250, loss_tot: 187.90204194065183
i: 54500, loss_tot: 205.80359384701586
i: 54750, loss_tot: 183.49203126790235
i: 55000, loss_tot: 218.31527184729464
i: 55250, loss_tot: 207.90454569417983
i: 55500, loss_tot: 223.5672454131278
i: 55750, loss_tot: 206.1351486755861
i: 56000, loss_tot: 223.97713837882503
i: 56250, loss_tot: 197.50786902117542
i: 56500, loss_tot: 239.7570540627511
i: 56750, loss_tot: 244.33578633099793
i: 57000, loss_tot: 277.9544284191914
i: 57250, loss_tot: 230.16044629218058
i: 57500, loss_tot: 264.22975816995836
i: 57750, loss_tot: 277.35292240699755
i: 58000, loss_tot: 239.65496104590594
i: 58250, loss_tot: 181.84221364041093
i: 58500, loss_tot: 244.8915189059172
i: 58750, loss_tot: 219.90682541863293
i: 59000, loss_tot: 193.96386425940784
i: 59250, loss_tot: 265.65226422782985
i: 59500, loss_tot: 196.30504738237244
i: 59750, loss_tot: 229.379449

i: 106500, loss_tot: 253.46463328302255
i: 106750, loss_tot: 232.4778765048599
i: 107000, loss_tot: 237.55550339414492
i: 107250, loss_tot: 199.33844076780215
i: 107500, loss_tot: 227.90935976129012
i: 107750, loss_tot: 212.662644226775
i: 108000, loss_tot: 200.689330088367
i: 108250, loss_tot: 204.69631796650035
i: 108500, loss_tot: 213.89729173250518
i: 108750, loss_tot: 214.47074522374896
i: 109000, loss_tot: 216.17597608039156
i: 109250, loss_tot: 261.8982407411188
i: 109500, loss_tot: 230.42022186422022
i: 109750, loss_tot: 194.53987273549194
i: 110000, loss_tot: 242.0785566044299
i: 110250, loss_tot: 186.47825439905458
i: 110500, loss_tot: 218.49419141830526
i: 110750, loss_tot: 232.53813643434552
i: 111000, loss_tot: 180.18275376212785
i: 111250, loss_tot: 258.1342973970529
i: 111500, loss_tot: 221.78341306778603
i: 111750, loss_tot: 185.02807533215963
i: 112000, loss_tot: 186.2398552709166
i: 112250, loss_tot: 226.5505964107241
i: 112500, loss_tot: 251.79410597045907
i: 112750,

i: 158250, loss_tot: 273.79737309615825
i: 158500, loss_tot: 248.39857039453753
i: 158750, loss_tot: 208.63610787773302
i: 159000, loss_tot: 316.11544759259186
i: 159250, loss_tot: 193.68333580251667
i: 159500, loss_tot: 210.07463427734382
i: 159750, loss_tot: 246.67457502349106
i: 160000, loss_tot: 253.46721022756654
i: 160250, loss_tot: 205.53194766984322
i: 160500, loss_tot: 191.24232087644734
i: 160750, loss_tot: 218.69996197019486
i: 161000, loss_tot: 228.9686958142108
i: 161250, loss_tot: 219.2850498936884
i: 161500, loss_tot: 264.9396471658862
i: 161750, loss_tot: 244.04483930122223
i: 162000, loss_tot: 271.2400824238178
i: 162250, loss_tot: 285.34641656265376
i: 162500, loss_tot: 214.04839830499157
i: 162750, loss_tot: 192.1898056708055
i: 163000, loss_tot: 197.22354000536316
i: 163250, loss_tot: 220.59410395575352
i: 163500, loss_tot: 252.3521079925273
i: 163750, loss_tot: 270.7512111997148
i: 164000, loss_tot: 228.71704006306743
i: 164250, loss_tot: 202.03034034657657
i: 1645

...

i: 576750, loss_tot: 262.9644492306281
i: 577000, loss_tot: 268.5376177310664
i: 577250, loss_tot: 189.55169617084786
i: 577500, loss_tot: 199.54723428823053
i: 577750, loss_tot: 242.69521252424457
i: 578000, loss_tot: 270.31080128835515
i: 578250, loss_tot: 197.70752793615802
i: 578500, loss_tot: 231.74930546165444
i: 578750, loss_tot: 241.2502410798148
i: 579000, loss_tot: 261.1486596222785
i: 579250, loss_tot: 210.3658348857169
i: 579500, loss_tot: 206.8673452442825
i: 579750, loss_tot: 187.77929957350017
i: 580000, loss_tot: 209.40448521762076
i: 580250, loss_tot: 197.9746032600222
i: 580500, loss_tot: 171.59750246102863
i: 580750, loss_tot: 208.41313536195085
i: 581000, loss_tot: 293.07122738131324
i: 581250, loss_tot: 192.96526285238332
i: 581500, loss_tot: 248.50760428451002
i: 581750, loss_tot: 197.31304269610075
i: 582000, loss_tot: 210.24056498820428
i: 582250, loss_tot: 242.51875421423466
i: 582500, loss_tot: 212.28686331769453
i: 582750, loss_tot: 237.9307080650609
i: 58300

i: 628500, loss_tot: 257.70720541239297
i: 628750, loss_tot: 198.93611151242908
i: 629000, loss_tot: 221.76173611864854
i: 629250, loss_tot: 171.97616132064277
i: 629500, loss_tot: 191.99172647357946
i: 629750, loss_tot: 303.6225040785955
i: 630000, loss_tot: 215.92240322075784
i: 630250, loss_tot: 263.3211314351205
i: 630500, loss_tot: 213.256870596895
i: 630750, loss_tot: 204.074246023288
i: 631000, loss_tot: 223.0984052023245
i: 631250, loss_tot: 216.0044294408965
i: 631500, loss_tot: 216.1279933447392
i: 631750, loss_tot: 242.80647651409294
i: 632000, loss_tot: 222.20492712593753
i: 632250, loss_tot: 294.89105797845536
i: 632500, loss_tot: 249.31122063362972
i: 632750, loss_tot: 254.70223384037257
i: 633000, loss_tot: 245.1876728801978
i: 633250, loss_tot: 207.58274408710383
i: 633500, loss_tot: 228.4439568282677
i: 633750, loss_tot: 278.3643546783371
i: 634000, loss_tot: 223.5838115686305
i: 634250, loss_tot: 224.8450546283461
i: 634500, loss_tot: 227.79137141697575
i: 634750, los

i: 680250, loss_tot: 235.7081776235478
i: 680500, loss_tot: 185.38965827423715
i: 680750, loss_tot: 199.45094057596987
i: 681000, loss_tot: 193.28871465354226
i: 681250, loss_tot: 205.82278908068926
i: 681500, loss_tot: 236.59931544080945
i: 681750, loss_tot: 220.92916982158087
i: 682000, loss_tot: 211.33308540992803
i: 682250, loss_tot: 174.6438211685809
i: 682500, loss_tot: 255.60148853124775
i: 682750, loss_tot: 205.09937022564466
i: 683000, loss_tot: 211.2707136230392
i: 683250, loss_tot: 257.3594926194951
i: 683500, loss_tot: 224.01556247266709
i: 683750, loss_tot: 219.68179311581716
i: 684000, loss_tot: 224.8729581945989
i: 684250, loss_tot: 240.55369103688395
i: 684500, loss_tot: 290.28361140673513
i: 684750, loss_tot: 231.9563404605447
i: 685000, loss_tot: 241.9770468857579
i: 685250, loss_tot: 232.04038588774623
i: 685500, loss_tot: 251.14558771303157
i: 685750, loss_tot: 227.97496338438702
i: 686000, loss_tot: 242.81701503974662
i: 686250, loss_tot: 204.222683371288
i: 686500

i: 732000, loss_tot: 242.1328905795864
i: 732250, loss_tot: 261.1886766362407
i: 732500, loss_tot: 200.52382099415422
i: 732750, loss_tot: 231.05283008014877
i: 733000, loss_tot: 235.14296671147866
i: 733250, loss_tot: 215.01136665513738
i: 733500, loss_tot: 254.73607313103042
i: 733750, loss_tot: 200.0551202340611
i: 734000, loss_tot: 181.37243620429973
i: 734250, loss_tot: 208.24793758187886
i: 734500, loss_tot: 200.17104453943233
i: 734750, loss_tot: 224.4338899835949
i: 735000, loss_tot: 186.53052984432202
i: 735250, loss_tot: 262.675071992349
i: 735500, loss_tot: 187.68043664181585
i: 735750, loss_tot: 210.18815524434504
i: 736000, loss_tot: 243.1117938328475
i: 736250, loss_tot: 286.387861172393
i: 736500, loss_tot: 196.5787823777643
i: 736750, loss_tot: 199.6312133125128
i: 737000, loss_tot: 185.5373161366023
i: 737250, loss_tot: 196.85254964414634
i: 737500, loss_tot: 233.19173012615997
i: 737750, loss_tot: 217.26445704846643
i: 738000, loss_tot: 211.74113845386398
i: 738250, l

i: 783750, loss_tot: 252.0923607951775
i: 784000, loss_tot: 236.01040406699292
i: 784250, loss_tot: 210.1004590171591
i: 784500, loss_tot: 177.3131952783402
i: 784750, loss_tot: 258.39702489283866
i: 785000, loss_tot: 208.47682026490568
i: 785250, loss_tot: 233.00928222691655
i: 785500, loss_tot: 227.41214659516467
i: 785750, loss_tot: 217.00513230351265
i: 786000, loss_tot: 254.49746485879018
i: 786250, loss_tot: 249.03764483374778
i: 786500, loss_tot: 231.95978598975
i: 786750, loss_tot: 222.3941777442385
i: 787000, loss_tot: 218.68668966561555
i: 787250, loss_tot: 266.04199720268
i: 787500, loss_tot: 218.64470007103645
i: 787750, loss_tot: 225.9509102216782
i: 788000, loss_tot: 247.47913627818227
i: 788250, loss_tot: 201.29257585125043
i: 788500, loss_tot: 230.26306699693203
i: 788750, loss_tot: 229.1577421088308
i: 789000, loss_tot: 201.26149965959144
i: 789250, loss_tot: 160.07087806764991
i: 789500, loss_tot: 251.21789539947233
i: 789750, loss_tot: 253.58057788004172
i: 790000, l

In [42]:
model.save_weights(f"{models_path}/model_v1")

In [ ]:
 
    true_ids_testing = []
    pred_ids_testing = []
    loss_tot_testing = 0.0
    for Xelem, ygen in zip(Xs_testing, ys_testing):
        pred_id, pred_momentum, _ = model(Xelem)
        true_ids_testing += [ygen[:, 0]]
        pred_ids_testing += [tf.argmax(pred_id, axis=-1).numpy()]
    true_ids_testing = np.concatenate(true_ids_testing)
    pred_ids_testing = np.concatenate(pred_ids_testing)

    acc = accuracy_score(true_ids, pred_ids)
    acc_testing = accuracy_score(true_ids_testing, pred_ids_testing)
    print("epoch={epoch} loss={loss:.2f} acc={acc:.4f}/{acc_testing:.4f}".format(epoch=i, loss=loss_tot, acc=acc, acc_testing=acc_testing))

In [ ]:

#eval_ds = eval_ds.padded_batch(batch_size, padded_shapes=([None,n_features],[None,n_seed_features],[None,1]),
#                                   padding_values=(0.,0.,0.),drop_remainder=True).repeat()
